<a href="https://colab.research.google.com/github/ChiriKamau/limaAI/blob/main/notebooks/leaf_model/yololeafmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Leaf model


In [1]:
!pip install ultralytics opencv-python -q
from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.9 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import os
import shutil
from PIL import Image as PILImage
from sklearn.model_selection import train_test_split



In [3]:
#CSV paths and their corresponding image directories (modify these as needed)
csv_image_mapping = [
    {
        'csv_path': '/content/drive/MyDrive/Tomato_dataset/Phone1_Leaves(batch3)/annotations/phone 1_leaves(batch3).csv',
        'image_dir': '/content/drive/MyDrive/Tomato_dataset/Phone1_Leaves(batch3)'
    },
    {'csv_path': '/content/drive/MyDrive/Tomato_dataset/Phone2_leaves/annotations/Phone2_leaves.csv',
        'image_dir': '/content/drive/MyDrive/Tomato_dataset/Phone2_leaves'
    },
     {'csv_path': '/content/drive/MyDrive/Tomato_dataset/phone1_leaves(batch1)/annotations/phone1_leaves(batch1).csv',
        'image_dir': '/content/drive/MyDrive/Tomato_dataset/phone1_leaves(batch1)'
    },
     {'csv_path': '/content/drive/MyDrive/Tomato_dataset/phone1_leaves(batch2)/annotations/phone1_leaves(batch2).csv',
        'image_dir': '/content/drive/MyDrive/Tomato_dataset/phone1_leaves(batch2)'
    }
    ]

In [4]:
# Get annotated image names from CSVs
annotated_images = set()
for mapping in csv_image_mapping:
    try:
        df = pd.read_csv(mapping['csv_path'])
        annotated_images.update(df['image_name'].unique())
    except Exception as e:
        print(f"Error reading {mapping['csv_path']}: {e}")

# Get all images from directories
all_images = set()
for mapping in csv_image_mapping:
    try:
        images = [f for f in os.listdir(mapping['image_dir']) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        all_images.update(images)
    except Exception as e:
        print(f"Error accessing {mapping['image_dir']}: {e}")

# Calculate annotated and unannotated counts
unannotated_images = all_images - annotated_images
print(f"Annotated images: {len(annotated_images)}")
print(f"Unannotated images: {len(unannotated_images)}")
if unannotated_images:
    print("Unannotated images:", unannotated_images)

Annotated images: 515
Unannotated images: 96
Unannotated images: {'IMG_20250906_124704.jpg', 'IMG_20250906_124803.jpg', 'IMG_20250906_125943.jpg', 'IMG_20250906_130336.jpg', 'IMG_20250906_124244.jpg', 'IMG_20250906_130034.jpg', 'IMG_20250906_124418.jpg', 'IMG_20250906_124909.jpg', 'IMG_20250906_124503.jpg', 'IMG_20250906_130027.jpg', 'IMG_20250906_130824.jpg', 'IMG_20250906_132222_HDR.jpg', 'IMG_20250906_143920_3.jpg', 'IMG_20250906_130557.jpg', 'IMG_20250906_124644.jpg', 'IMG_20250906_130331.jpg', 'IMG_20250906_125136.jpg', 'IMG_20250906_130508.jpg', 'IMG_20250906_131708.jpg', 'IMG_20250906_132738.jpg', 'IMG_20250906_125403.jpg', 'IMG_20250906_131711.jpg', 'IMG_20250906_143119_HDR.jpg', 'IMG_20250906_125756.jpg', 'IMG_20250906_125654.jpg', 'IMG_20250906_131014.jpg', 'IMG_20250906_130652.jpg', 'IMG_20250906_125228.jpg', 'IMG_20250906_125120.jpg', 'IMG_20250906_130304.jpg', 'IMG_20250906_130321.jpg', 'IMG_20250906_130925.jpg', 'IMG_20250906_143429_HDR.jpg', 'IMG_20250906_124404.jpg', 'I

In [6]:
# Check annotated vs unannotated per folder
for mapping in csv_image_mapping:
    csv_path = mapping['csv_path']
    image_dir = mapping['image_dir']
    folder_name = os.path.basename(image_dir)

    try:
        # Load annotated image names
        df = pd.read_csv(csv_path)
        annotated = set(df['image_name'].unique())
    except Exception as e:
        print(f"Error reading CSV for {folder_name}: {e}")
        annotated = set()

    try:
        # Get all images
        all_imgs = {f for f in os.listdir(image_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))}
    except Exception as e:
        print(f"Error accessing images in {folder_name}: {e}")
        all_imgs = set()

    # Compare
    unannotated = all_imgs - annotated

    print(f"\n Folder: {folder_name}")
    print(f"  Total images:       {len(all_imgs)}")
    print(f"  Annotated images:   {len(annotated)}")
    print(f"  Unannotated images: {len(unannotated)}")

    if unannotated:
        print("   Unannotated samples:", list(unannotated)[:5], "..." if len(unannotated) > 5 else "")


 Folder: Phone1_Leaves(batch3)
  Total images:       113
  Annotated images:   96
  Unannotated images: 17
   Unannotated samples: ['IMG_20250906_124704.jpg', 'IMG_20250906_124503.jpg', 'IMG_20250906_125120.jpg', 'IMG_20250906_125223.jpg', 'IMG_20250906_125136.jpg'] ...

 Folder: Phone2_leaves
  Total images:       218
  Annotated images:   191
  Unannotated images: 27
   Unannotated samples: ['IMG_20250906_124552.jpg', 'IMG_20250906_143920.jpg', 'IMG_20250906_143429_HDR.jpg', 'IMG_20250906_124448_HDR.jpg', 'IMG_20250906_124404.jpg'] ...

 Folder: phone1_leaves(batch1)
  Total images:       150
  Annotated images:   139
  Unannotated images: 11
   Unannotated samples: ['IMG_20250906_131450.jpg', 'IMG_20250906_132850.jpg', 'IMG_20250906_131708.jpg', 'IMG_20250906_132820.jpg', 'IMG_20250906_132738.jpg'] ...

 Folder: phone1_leaves(batch2)
  Total images:       150
  Annotated images:   106
  Unannotated images: 44
   Unannotated samples: ['IMG_20250906_130304.jpg', 'IMG_20250906_130321.

In [8]:
dfs = []
for mapping in csv_image_mapping:
    try:
        df = pd.read_csv(mapping['csv_path'])
        df['image_dir'] = mapping['image_dir']  # Add image_dir column
        dfs.append(df)
    except Exception as e:
        print(f"Error reading {mapping['csv_path']}: {e}")
        raise

# Concatenate DataFrames in memory for processing
data = pd.concat(dfs, ignore_index=True)

# Ensure correct columns
expected_columns = ['image_name', 'label_name', 'bbox_x', 'bbox_y',
                    'bbox_width', 'bbox_height', 'image_width',
                    'image_height', 'image_dir']
data = data[expected_columns]

# Check for missing or invalid labels
print("Missing label_name values:", data['label_name'].isna().sum())
print("Unique label_name values:", data['label_name'].unique())

data = data.dropna(subset=['label_name'])
data = data[data['label_name'].apply(lambda x: isinstance(x, str))]

# ------------------------------
# MULTICLASS FIX (exact labels)
# ------------------------------
# Use exact label_name as the class
data['class_name'] = data['label_name'].astype(str).str.strip()

print("Unique multiclass labels:", data['class_name'].unique())

# Class distribution
print("Class distribution (multiclass):")
print(data['class_name'].value_counts())

# Validate bounding boxes
invalid_bboxes = data[
    (data['bbox_width'] <= 0) |
    (data['bbox_height'] <= 0) |
    (data['bbox_x'] < 0) |
    (data['bbox_y'] < 0) |
    (data['bbox_x'] + data['bbox_width'] > data['image_width']) |
    (data['bbox_y'] + data['bbox_height'] > data['image_height'])
]
if not invalid_bboxes.empty:
    print("Invalid bounding boxes:", invalid_bboxes[['image_name', 'bbox_x', 'bbox_y', 'bbox_width', 'bbox_height']])
    data = data[~data.index.isin(invalid_bboxes.index)]

# Train/val split
unique_filenames = data['image_name'].unique()
train_filenames, val_filenames = train_test_split(unique_filenames, test_size=0.2, random_state=42)

# Create dataset directories
dataset_dir = '/content/dataset'
images_train_dir = os.path.join(dataset_dir, 'images', 'train')
images_val_dir = os.path.join(dataset_dir, 'images', 'val')
labels_train_dir = os.path.join(dataset_dir, 'labels', 'train')
labels_val_dir = os.path.join(dataset_dir, 'labels', 'val')

os.makedirs(images_train_dir, exist_ok=True)
os.makedirs(images_val_dir, exist_ok=True)
os.makedirs(labels_train_dir, exist_ok=True)
os.makedirs(labels_val_dir, exist_ok=True)

# ---------------------------------
# MULTICLASS CLASS MAP (EXACT)
# ---------------------------------
unique_classes = sorted(data['class_name'].unique())
class_map = {cls: i for i, cls in enumerate(unique_classes)}

print("CLASS MAP (exact names):")
print(class_map)


Missing label_name values: 0
Unique label_name values: ['Flowers' 'Leaf_spot' 'Y.leaf_curl' 'Wilt' 'Early_blight' 'Late_blight'
 'Late _blight' 'Pests' 'Early-blight' 'Powdery_mildew' 'flowers']
Unique multiclass labels: ['Flowers' 'Leaf_spot' 'Y.leaf_curl' 'Wilt' 'Early_blight' 'Late_blight'
 'Late _blight' 'Pests' 'Early-blight' 'Powdery_mildew' 'flowers']
Class distribution (multiclass):
class_name
Flowers           602
Y.leaf_curl       447
flowers           163
Powdery_mildew    121
Wilt              114
Early_blight      105
Late_blight        94
Leaf_spot          79
Late _blight       72
Pests              36
Early-blight       11
Name: count, dtype: int64
CLASS MAP (exact names):
{'Early-blight': 0, 'Early_blight': 1, 'Flowers': 2, 'Late _blight': 3, 'Late_blight': 4, 'Leaf_spot': 5, 'Pests': 6, 'Powdery_mildew': 7, 'Wilt': 8, 'Y.leaf_curl': 9, 'flowers': 10}


In [9]:
# Create YOLO label file (MULTICLASS VERSION)
def create_yolo_label(filename, split_dir, is_train=True):
    annotations = data[data['image_name'] == filename]
    if annotations.empty:
        print(f"No annotations for {filename}")
        return False

    image_dir = annotations['image_dir'].iloc[0]
    image_path = os.path.join(image_dir, filename)
    if not os.path.exists(image_path):
        print(f"Image not found: {image_path}")
        return False

    width = annotations['image_width'].iloc[0]
    height = annotations['image_height'].iloc[0]

    # Choose train or val folder
    dest_image_dir = images_train_dir if is_train else images_val_dir
    dest_image_path = os.path.join(dest_image_dir, filename)
    shutil.copy(image_path, dest_image_path)

    # YOLO label file path
    label_path = os.path.join(
        labels_train_dir if is_train else labels_val_dir,
        filename.replace('.jpg', '.txt')
    )

    valid_annotations = 0

    with open(label_path, 'w') as f:
        for _, ann in annotations.iterrows():

            # ---------------------------------------------------
            # MULTICLASS: use exact label_name → class_name → class_id
            # ---------------------------------------------------
            cls = class_map[ann['class_name']]

            # Normalize bbox
            x_center = (ann['bbox_x'] + ann['bbox_width'] / 2) / width
            y_center = (ann['bbox_y'] + ann['bbox_height'] / 2) / height
            w = ann['bbox_width'] / width
            h = ann['bbox_height'] / height

            # Valid YOLO format?
            if 0 <= x_center <= 1 and 0 <= y_center <= 1 and 0 < w <= 1 and 0 < h <= 1:
                f.write(f"{cls} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")
                valid_annotations += 1
            else:
                print(f"Skipping invalid bbox for {filename}: {x_center}, {y_center}, {w}, {h}")

    print(f"Created label file: {label_path} with {valid_annotations} annotations")
    return valid_annotations > 0


In [10]:
# Create YOLO labels for train and val
train_success = []
val_success = []

# Train set
for filename in train_filenames:
    train_success.append(create_yolo_label(filename, 'train', is_train=True))

# Val set
for filename in val_filenames:
    val_success.append(create_yolo_label(filename, 'val', is_train=False))

# Verify dataset
print(f"Train images: {len(os.listdir(images_train_dir))} in {images_train_dir}")
print(f"Val images: {len(os.listdir(images_val_dir))} in {images_val_dir}")
print(f"Train labels: {len(os.listdir(labels_train_dir))} in {labels_train_dir}")
print(f"Val labels: {len(os.listdir(labels_val_dir))} in {labels_val_dir}")
print(f"Dataset prepared with {sum(train_success)} train images and {sum(val_success)} val images with valid annotations.")


Created label file: /content/dataset/labels/train/IMG_20250906_132906.txt with 2 annotations
Created label file: /content/dataset/labels/train/IMG_20250906_133734.txt with 4 annotations
Created label file: /content/dataset/labels/train/IMG_20250906_130903.txt with 2 annotations
Created label file: /content/dataset/labels/train/IMG_20250906_143850.txt with 4 annotations
Created label file: /content/dataset/labels/train/IMG_20250906_130318.txt with 2 annotations
Created label file: /content/dataset/labels/train/IMG_20250906_132707.txt with 4 annotations
Created label file: /content/dataset/labels/train/IMG_20250906_143834_HDR.txt with 9 annotations
Created label file: /content/dataset/labels/train/IMG_20250906_134410.txt with 5 annotations
Created label file: /content/dataset/labels/train/IMG_20250906_124738.txt with 5 annotations
Created label file: /content/dataset/labels/train/IMG_20250906_143539.txt with 5 annotations
Created label file: /content/dataset/labels/train/IMG_20250906_133

In [11]:
# Check a sample label file
sample_label = os.path.join(labels_train_dir, 'IMG_20250906_130900.txt')
if os.path.exists(sample_label):
    with open(sample_label, 'r') as f:
        print(f"Sample label file {sample_label}:\n{f.read()}")

In [12]:
import os
import yaml  # optional, but recommended for proper YAML formatting

# Path to dataset
yaml_path = '/content/data.yaml'

# Prepare YAML content
yaml_content = {
    'train': os.path.join(dataset_dir, 'images', 'train'),
    'val': os.path.join(dataset_dir, 'images', 'val'),
    'nc': len(class_map),
    'names': [cls for cls, idx in sorted(class_map.items(), key=lambda x: x[1])]
}

# Write YAML file
with open(yaml_path, 'w') as f:
    yaml.dump(yaml_content, f, default_flow_style=False)

# Verify
if os.path.exists(yaml_path):
    print(f"data.yaml created at {yaml_path}")
    with open(yaml_path, 'r') as f:
        print("data.yaml content:\n", f.read())
else:
    print(f"Failed to create data.yaml at {yaml_path}")


data.yaml created at /content/data.yaml
data.yaml content:
 names:
- Early-blight
- Early_blight
- Flowers
- Late _blight
- Late_blight
- Leaf_spot
- Pests
- Powdery_mildew
- Wilt
- Y.leaf_curl
- flowers
nc: 11
train: /content/dataset/images/train
val: /content/dataset/images/val



In [ ]:
from ultralytics import YOLO
import os

# Verify data.yaml exists
yaml_path = '/content/data.yaml'
if not os.path.exists(yaml_path):
    raise FileNotFoundError(f"Error: {yaml_path} does not exist")

# Load pretrained YOLOv8 medium model
model = YOLO('yolov8m.pt')

# -------------------------------
# Optional: Compute class weights for multiclass
# -------------------------------
class_counts = data['class_name'].value_counts()
total = class_counts.sum()
# Compute weight = total / (num_classes * class_count)
num_classes = len(class_counts)
class_weights_list = [total / (num_classes * class_counts.get(cls, 1)) for cls in class_counts.index]

print("Multiclass class counts:\n", class_counts)
print("Computed class weights:", class_weights_list)

# -------------------------------
# Train YOLOv8
# -------------------------------
results = model.train(
    data=yaml_path,
    epochs=50,
    imgsz=320,
    batch=16,
    augment=True,
    hsv_h=0.4,
    hsv_s=0.9,
    hsv_v=0.7,
    fliplr=0.5,
    flipud=0.5,
    scale=0.8,
    translate=0.4,
    mosaic=1.0,
    mixup=0.4,
    patience=30,
    lr0=0.0003,
    iou=0.5,
    max_det=90,
    # class_weights=class_weights_list  # Optional: YOLOv8 supports list of class weights
)

print("Training completed.")

# Save path
save_path = '/content/drive/MyDrive/Tomato_dataset/models/yolov8m_trained(v3).pt'
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# Save the trained model
try:
    model.save(save_path)
    print(f"Model saved to {save_path}")
except Exception as e:
    print(f"Error saving model: {e}")
    raise


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Multiclass class counts:
 class_name
Flowers           602
Y.leaf_curl       447
flowers           163
Powdery_mildew    121
Wilt              114
Early_blight      105
Late_blight        94
Leaf_spot          79
Late _blight       72
Pests              36
Early-blight       11
Name: count, dtype: int64
Computed class weights: [np.float64(0.27846572032618544), np.float64(0.37502542200528777), np.float64(1.0284439486893475), np.float64(1.3854244928625095), np.float64(1.470494417862839), np.float64(1.5965367965367965), np.float64(1.7833655705996132), np.float64(2.1219792865362486), np.float64(2.3282828282828283), np.float64(4.656565656565657), np.float64(15.239669421487603)]
Ultraly